After completing this tutorial, you will know:

- Dimensionality reduction involves reducing the number of input variables or columns in modeling data.
- PCA is a technique from linear algebra that can be used to automatically perform dimensionality reduction.
- How to evaluate predictive models that use a PCA projection as input and make predictions with new raw data.

### Tutorial Overview
This tutorial is divided into three parts; they are:

 - Dimensionality Reduction and PCA
 - PCA Scikit-Learn API
 - Worked Example of PCA for Dimensionality Reduction

In [ ]:
# test classification dataset
from sklearn.datasets import make_classification

# define dataset
X, y = make_classification(
    n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7
)

# summarize the dataset
print(X.shape, y.shape)

In [ ]:
# evaluate pca with logistic regression algorithm for classification
from numpy import mean, std
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

# define dataset
X, y = make_classification(
    n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7
)

# define the pipeline
steps = [("pca", PCA(n_components=10)), ("m", LogisticRegression())]
model = Pipeline(steps=steps)

# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(
    model, X, y, scoring="accuracy", cv=cv, n_jobs=-1, error_score="raise"
)

# report performance
print("Accuracy: %.3f (%.3f)" % (mean(n_scores), std(n_scores)))

A better approach is to evaluate the same transform and model with different numbers of input features and choose the number of features (amount of dimensionality reduction) that results in the best average performance.

The example below performs this experiment and summarizes the mean classification accuracy for each configuration.

In [ ]:
# compare pca number of components with logistic regression algorithm for classification
from matplotlib import pyplot
from numpy import mean, std
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline


# get the dataset
def get_dataset():
    X, y = make_classification(
        n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7
    )
    return X, y


# get a list of models to evaluate
def get_models():
    models = dict()
    for i in range(1, 21):
        steps = [("pca", PCA(n_components=i)), ("m", LogisticRegression())]
        models[str(i)] = Pipeline(steps=steps)
    return models


# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(
        model, X, y, scoring="accuracy", cv=cv, n_jobs=-1, error_score="raise"
    )
    return scores


# define dataset
X, y = get_dataset()

# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print(">%s %.3f (%.3f)" % (name, mean(scores), std(scores)))

# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.xticks(rotation=45)
pyplot.show()

We may choose to use a PCA transform and logistic regression model combination as our final model.

This involves fitting the Pipeline on all available data and using the pipeline to make predictions on new data. Importantly, the same transform must be performed on this new data, which is handled automatically via the Pipeline.

The example below provides an example of fitting and using a final model with PCA transforms on new data.

In [ ]:
# make predictions using pca with logistic regression
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# define dataset
X, y = make_classification(
    n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7
)

# define the model
steps = [("pca", PCA(n_components=15)), ("m", LogisticRegression())]
model = Pipeline(steps=steps)

# fit the model on the whole dataset
model.fit(X, y)

# make a single prediction
row = [
    [
        0.2929949,
        -4.21223056,
        -1.288332,
        -2.17849815,
        -0.64527665,
        2.58097719,
        0.28422388,
        -7.1827928,
        -1.91211104,
        2.73729512,
        0.81395695,
        3.96973717,
        -2.66939799,
        3.34692332,
        4.19791821,
        0.99990998,
        -0.30201875,
        -4.43170633,
        -2.82646737,
        0.44916808,
    ]
]
yhat = model.predict(row)
print("Predicted Class: %d" % yhat[0])